In [1]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, KnownRecord, MergeSearchArtist, ConcatRawData
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.genius import MusicDBParams, RawAPIData, MusicDBIO
from functools import partial
from os import getpid

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
apiio = RawAPIData()
db = mpar.db

In [3]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRecord = DownloadRecord(db=db, name="DownloadArtist", rTypes=["Index"])
downloadArtistSongRecord = DownloadRecord(db=db, name="DownloadArtistSong", rTypes=["Index"])
downloadAlbumRecord = DownloadRecord(db=db, name="DownloadAlbum", rTypes=["Index"])
knownAlbumRecord = KnownRecord(db=db, name="Album")
downloadSongRecord = DownloadRecord(db=db, name="DownloadSong", rTypes=["Index"])
knownSongRecord = KnownRecord(db=db, name="Song")
allArtists = partial(dbio.rdio.getData, "SearchArtist")
knownArtists = dbio.rdio.getSummaryNameData

DownloadRecord(db=Genius, name=SearchArtist, rTypes=['Index', 'Data'])
DownloadRecord(db=Genius, name=DownloadArtist, rTypes=['Index'])
DownloadRecord(db=Genius, name=DownloadArtistSong, rTypes=['Index'])
DownloadRecord(db=Genius, name=DownloadAlbum, rTypes=['Index'])
DownloadRecord(db=Genius, name=Album)
DownloadRecord(db=Genius, name=DownloadSong, rTypes=['Index'])
DownloadRecord(db=Genius, name=Song)


In [ ]:
if False:
    from utils import FileInfo
    import shutil
    db = dbio.params.db
    dbdir = dbio.params.dir
    nameMap = {"MasterArtists": "SearchArtist", "LocalArtists": "DownloadArtist", "LocalAlbums": "DownloadAlbum"}
    nameMap = nameMap | {"LocalArtistSongs": "DownloadArtistSong", "LocalSongs": "DownloadSong", "LocalAlbums": "DownloadAlbum"}
    dinfo = DirInfo(f"/Users/tgadfort/anaconda3/envs/post/pandb/musicdb/{db}")
    
    cpMap = {}
    for origname, newname in nameMap.items():
        cpMap[f"{dbdir}SearchedFor{origname}"] = f"{db}-{newname}-Index"
        cpMap[f"{dbdir}SearchedFor{origname}Data"] = f"{db}-{newname}-Data"
        cpMap[f"{dbdir}SearchedFor{origname}Errors"] = f"{db}-{newname}-Error"
    assert dinfo.exists(), f"{dinfo} does not exist!"
    for srcFile, dstFile in cpMap.items():
        srcFile = dinfo.join(f"{srcFile}.p")
        dstFile = dinfo.join(f"{dstFile}.p")
        if srcFile.exists():
            srcFile.mvFile(dstFile)

In [ ]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRecord.info()
downloadArtistSongRecord.info()
downloadSongRecord.info()
knownSongRecord.info()
downloadAlbumRecord.info()
knownAlbumRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

# Search For New Artists

In [ ]:
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return DataFrame()
    newData = concat([Series(s) for s in newData.values()])
    newData.index = newData.index.astype(str)
    newData = newData[~newData.index.duplicated()]
    return newData

def mergeNewData(searchArtistRecord):
    searchArtistRecord.load(verbose=False)
    newData = getNewData(searchArtistRecord)
    msr = MergeSearchArtist(db)
    msr.mergeLocal(searchArtistRecord, newData, test=False)

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = PanDBIO().getUniqueArtistNames()
mergeNewData(searchArtistRecord)
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.map(searchArtistRecord.isKnown)]

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 250000000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 5:
        print("Stopping due to 5 consecutive errors")
        break

    try:
        response = apiio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        isError(artistName, nErrors, 10, error)
        continue

    if not isinstance(response, dict):
        isError(artistName, nErrors, 3.5, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    apiio.sleep(2.5)
    n += 1
        
    if n % 25 == 0:
        ts.update(n=n)
        searchArtistRecord.save()
        apiio.wait(10.0)
        if tt.isFinished():
            break
    
    if n >= maxN:
        print("Breaking after {0} downloads...".format(maxN))
        break

ts.stop()
searchArtistRecord.save()

# Download Artist Data

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=True)
apiio = RawAPIData(debug=False)
knownNames = DataFrame(allArtists()).rename(columns={0: "Name"})
knownNames['ModVal'] = knownNames.index.map(mv.getModVal)
downloadedNames = Series(downloadArtistRecord.getIndex())
availableNames = knownNames[~knownNames.index.isin(downloadedNames.index)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby("ModVal")})

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadedNames.shape[0]}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del downloadedNames
del knownNames

In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistID, artistName, nErrors, sleeptime, error=None):
    downloadArtistRecord.setError(index=artistID)
    print(f"Download Error ==> {artistName}: {error}")
    nErrors.append(artistID)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 250000000
stop = False
nErrors = []
for modVal,modValData in artistNamesToGet.items():
    #modVal = groupModVal[0]
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(artistID,artistName) in enumerate(modValData['Name'].items()):    
        if downloadArtistRecord.isKnown(artistID):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break
    
        print(f"{modVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")
        try:
            response = apiio.getArtistData(artistID=artistID, artistName=artistName)        
        except Exception as error:
            isError(artistID, artistName, nErrors, 5, error)
            continue
    
        if not isinstance(response, dict):
            isError(artistID, artistName, nErrors, 3.5, "NotDict")
            continue

        
        nErrors = []
        downloadArtistRecord.setIndex(index=artistID)
        dbio.rdio.saveData("RawArtist", modVal, artistID, data=response)
        apiio.sleep(2.0)
        n += 1
            
        if n % 25 == 0:
            ts.update(n=n)
            downloadArtistRecord.save()
            apiio.wait(15.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print(f"Breaking after {maxN} downloads...")
            stop = True
            break

ts.stop()
downloadArtistRecord.save()

In [ ]:
downloadArtistRecord.save()

In [ ]:
crd = ConcatRawData(db=dbio.db, dType="Artist")
#crd.concat()
#crd.merge(test=False)
#crd.remove(force=True)

# Download Artist Songs Data

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = DataFrame(allArtists()).rename(columns={0: "Name"})
knownNames['ModVal'] = knownNames.index.map(mv.getModVal)
downloadedNames = Series(downloadArtistSongRecord.getIndex())
availableNames = knownNames[~knownNames.index.isin(downloadedNames.index)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby("ModVal")})

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadedNames.shape[0]}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del downloadedNames
del knownNames

In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistID, artistName, nErrors, sleeptime, error=None):
    downloadArtistSongRecord.setError(index=artistID)
    print(f"Download Error ==> {artistName}: {error}")
    nErrors.append(artistID)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 2500000
stop = False
nErrors = []
for modVal,modValData in artistNamesToGet.items():
    #modVal = groupModVal[0]
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(artistID,artistName) in enumerate(modValData['Name'].items()):    
        if downloadArtistSongRecord.isKnown(artistID):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break
    
        print(f"{modVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")
        try:
            response = apiio.getArtistSongsData(artistID=artistID, artistName=artistName)        
        except Exception as error:
            isError(artistID, artistName, nErrors, 5, error)
            continue
    
        if not isinstance(response, dict):
            isError(artistID, artistName, nErrors, 3.5, "NotDict")
            continue

        
        nErrors = []
        downloadArtistSongRecord.setIndex(index=artistID)
        dbio.rdio.saveData("RawArtistSong", modVal, artistID, data=response)
        apiio.sleep(2.0)
        n += 1
            
        if n % 25 == 0:
            ts.update(n=n)
            downloadArtistSongRecord.save()
            apiio.wait(15.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print(f"Breaking after {maxN} downloads...")
            stop = True
            break

ts.stop()
downloadArtistSongRecord.save()

In [ ]:
downloadArtistSongRecord.save()

In [ ]:
crd = ConcatRawData(db=dbio.db, dType="ArtistSong")
#crd.concat()
#crd.merge(test=False)
crd.remove(force=True)

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = dbio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

# Download Songs

In [ ]:
dbio = MusicDBIO(verbose=False,local=False,mkDirs=False)
artistSongs = concat([dbio.rdio.getData("ModValArtistSong", modVal)[['dbid', 'name']] for modVal in range(100)])
artistSongs = artistSongs[~artistSongs['dbid'].duplicated()].rename(columns={"dbid": "SongID", "name": "SongName"})
def getaid(idx):
    retval = idx.split("-")[1][3:]
    return retval
artistSongs['ArtistID'] = artistSongs.index.map(getaid)
artistSongs.index = artistSongs['SongID']
artistSongs.index.name = ""
artistSongs = artistSongs.drop("SongID", axis=1)
data = artistSongs['SongName'].to_dict()
knownSongRecord.recordData['Data'] = data
knownSongRecord.save()

In [7]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = DataFrame(Series(knownSongRecord.getData(), name="SongName"))
knownNames['ModVal'] = knownNames.index.map(mv.getModVal)
availableNames = knownNames[~knownNames.index.map(downloadSongRecord.isKnown)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby("ModVal")})

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadSongRecord.numKnown()}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del knownNames

# Genius Search Results (PID=26046)
#   Available Names:     6268321
#   Known Artist Names:  6011726
#   Artist Names To Get: 256619


In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(songID, songName, nErrors, sleeptime, error=None):
    downloadSongRecord.setError(index=songID)
    print(f"Download Error ==> {songName}: {error}")
    nErrors.append(songID)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 2500000
stop = False
nErrors = []
for modVal,modValData in artistNamesToGet.items():
    #modVal = groupModVal[0]
    if stop is True:
        break
    N = modValData.shape[0]
    for i,(songID,songName) in enumerate(modValData['SongName'].items()):    
        if downloadSongRecord.isKnown(songID):
            continue
    
        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break
    
        print(f"{modVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")
        
        try:
            response = apiio.getSongData(songID=songID, songName=songName)        
        except Exception as error:
            isError(songID, songName, nErrors, 5, error)
            continue
    
        if not isinstance(response, dict):
            isError(songID, songName, nErrors, 3.5, "NotDict")
            continue

        
        nErrors = []
        downloadSongRecord.setIndex(index=songID)
        dbio.rdio.saveData("RawSong", modVal, songID, data=response)
        apiio.sleep(2.0)
        n += 1
            
        if n % 5 == 0:
            if tt.isFinished():
                stop = True
                break
            
        if n % 100 == 0:
            ts.update(n=n)
            downloadSongRecord.save()
            apiio.wait(15.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print(f"Breaking after {maxN} downloads...")
            stop = True
            break

ts.stop()
downloadSongRecord.save()

 [Getting Genius ArtistIDs] | StartTime 2023-12-05 14:45:14
   ====> Terminate Time Set To 2023-12-05 19:00:00 <====
   ====> Will Terminate Process 4 Hour and 14 Minute From Now
79       | 0        | 657      | 0        | Getting Song Info For Stall Stickers (8606979)                                     True
79       | 1        | 657      | 1        | Getting Song Info For Samo 3Aleko - سامو عليكو (8848779)                           True
79       | 2        | 657      | 2        | Getting Song Info For Kule 8 (9199679)                                             True
79       | 3        | 657      | 3        | Getting Song Info For Wild Life (8479179)                                          True
79       | 4        | 657      | 4        | Getting Song Info For When You’ll Die (9195379)                                    True
79       | 5        | 657      | 5        | Getting Song Info For Autumn’s Ending (3582479)                                    True
79       | 6        | 657    

Waiting:   0%|          | 0/150 [00:00<?, ?it/s]

79       | 100      | 657      | 100      | Getting Song Info For Die Stem van Tarvit (8830479)                                True
79       | 101      | 657      | 101      | Getting Song Info For Kau Tak Sendiri (4138379)                                    True
79       | 102      | 657      | 102      | Getting Song Info For Mesec žut (9357179)                                          True
79       | 103      | 657      | 103      | Getting Song Info For я странный (i’m weird) (8534179)                             True
79       | 104      | 657      | 104      | Getting Song Info For Constipation (5914579)                                       True
79       | 105      | 657      | 105      | Getting Song Info For Trepak (9480679)                                             True
79       | 106      | 657      | 106      | Getting Song Info For Weather (5408079)                                            True
79       | 107      | 657      | 107      | Getting Song Info For When I Die

Waiting:   0%|          | 0/150 [00:00<?, ?it/s]

79       | 200      | 657      | 200      | Getting Song Info For SckrewBall Flow (8685579)                                    True
79       | 201      | 657      | 201      | Getting Song Info For ITSUZO - Sexually attracted (6894979)                        True
79       | 202      | 657      | 202      | Getting Song Info For Celestial flowers (9559879)                                  True
79       | 203      | 657      | 203      | Getting Song Info For Maria Dipietro (6521879)                                     True
79       | 204      | 657      | 204      | Getting Song Info For 3500 Sacrifice (8979479)                                     True
79       | 205      | 657      | 205      | Getting Song Info For Backseat (8797879)                                           True
79       | 206      | 657      | 206      | Getting Song Info For Bitches (8273579)                                            True
79       | 207      | 657      | 207      | Getting Song Info For Do Synths 

...

80       | 243      | 6606     | 900      | Getting Song Info For Last Vestiges (8458380)                                      True
80       | 244      | 6606     | 901      | Getting Song Info For BU Sermon 1 (8806980)                                        True
80       | 245      | 6606     | 902      | Getting Song Info For Running (2980480)                                            True
80       | 246      | 6606     | 903      | Getting Song Info For Soon Summer - Remastered (5544080)                           True
80       | 247      | 6606     | 904      | Getting Song Info For Peppa Pig vs Simon Cowell (8940380)                          True
80       | 248      | 6606     | 905      | Getting Song Info For Estar Ao Pé de Ti (Encore Sessions) (8837180)                True
80       | 249      | 6606     | 906      | Getting Song Info For Save Point (8416580)                                         True
80       | 250      | 6606     | 907      | Getting Song Info For Weakness (

Waiting:   0%|          | 0/150 [00:00<?, ?it/s]

80       | 343      | 6606     | 1000     | Getting Song Info For The Other Side (338780)                                      True
80       | 344      | 6606     | 1001     | Getting Song Info For Mox Kitty (9574480)                                          True
80       | 345      | 6606     | 1002     | Getting Song Info For The End (9121980)                                            True
80       | 346      | 6606     | 1003     | Getting Song Info For How The World Goes (7803980)                                 True
80       | 347      | 6606     | 1004     | Getting Song Info For Cookies (7739380)                                            True
80       | 348      | 6606     | 1005     | Getting Song Info For High Ass Wave (9642980)                                      True
80       | 349      | 6606     | 1006     | Getting Song Info For DISCONNECTION (9230180)                                      True
80       | 350      | 6606     | 1007     | Getting Song Info For Cruise Jam

Waiting:   0%|          | 0/150 [00:00<?, ?it/s]

80       | 443      | 6606     | 1100     | Getting Song Info For JUMP (7289380)                                               True
80       | 444      | 6606     | 1101     | Getting Song Info For Lätt bagage (8661480)                                        True
80       | 445      | 6606     | 1102     | Getting Song Info For Maliblue (9616780)                                           True
80       | 446      | 6606     | 1103     | Getting Song Info For Contortion dream (9266680)                                   True
80       | 447      | 6606     | 1104     | Getting Song Info For WTW? (4714280)                                               True
80       | 448      | 6606     | 1105     | Getting Song Info For DISSENTIENT (6361280)                                        True
80       | 449      | 6606     | 1106     | Getting Song Info For Decir Adiós (9026480)                                        True
80       | 450      | 6606     | 1107     | Getting Song Info For Tutto ciò 

In [19]:

downloadSongRecord.save()

*****************************************************************************************************************************
*                                                    Saving DownloadSong                                                    *
*****************************************************************************************************************************
  Saving Index [5819426] ... Done
  Saving Error [0] ... Done


In [6]:
dbio = MusicDBIO(local=True, mkDirs=False)
for modVal in range(1,100):
    crd = ConcatRawData(db=dbio.db, dType="Song")
    crd.concat(modVal=modVal)
    crd.merge(modVal=modVal, test=False)
    crd.remove(modVal=modVal, force=True)

  ==> Making Directory: /Users/tgadfort/Music/Discog/artists-genius/tmp
ConcatRawData(db=Genius, dtype=Song
*****************************************************************************************************************************
*                                                  Concat Genius Song Data                                                  *
*****************************************************************************************************************************
DirInfo(path=/Users/tgadfort/Music/Discog/artists-genius/1/songs) 	 6616
****************************************************************************************************************************
*                                                 Merging Genius Song Data                                                 *
****************************************************************************************************************************
******************************************************************

In [5]:
crd.remove(modVal=modVal, force=True)

*****************************************************************************************************************************
*                                                 Removing Genius Song Data                                                 *
*****************************************************************************************************************************
  ==> Removing Directory: /Users/tgadfort/Music/Discog/artists-genius/tmp
